# Quantum Information Science HW P2
### Harris A. Ransom
### 11/11/24

## Question 1

Each Tanner graph is considered a **bipartite** graph, since it can be divided into two sets of nodes that do not have any intraconnecting edges within their respective sets.
For a random (3;6) Tanner graph, we can compute: 
- $\lambda (x) = $
- $\rho (x) = $

## Question 2

### Question 2a

### Question 2b